In [2]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

In [3]:
class graph:
    def __init__(self, x, y, theta):
        self.x = x
        self.y = y
        self.x_points = np.array([x])
        self.y_points = np.array([y])
        self.theta = theta

    def move_forward(self, distance):
        self.x += distance * np.cos(np.radians(self.theta))
        self.y += distance * np.sin(np.radians(self.theta))
        self.x_points = np.append(self.x_points, self.x)
        self.y_points = np.append(self.y_points, self.y)

    # Function to turn by a certain angle (degrees)
    def turn(self, degrees):
        self.theta += degrees


In [9]:

def calculate_error(car, track):
    diff = np.sin(np.radians(car.theta + 90)) * (track.x_points - car.x) - np.cos(np.radians(car.theta + 90)) * (track.y_points - car.y)
    crossings = np.where(np.diff(np.sign(diff)))[0]
    if crossings.size == 0:
        return 0
    
    distances = (track.x_points[crossings] - car.x) ** 2 + (track.y_points[crossings] - car.y) ** 2
    shortest_distance_index = np.argmin(distances)
    shortest_distance = distances[shortest_distance_index]

    if np.abs(shortest_distance) > 1:
        return 0
    
    if track.x_points[crossings][shortest_distance_index] > car.x:
        shortest_distance = -shortest_distance
    return shortest_distance

def PID(kp=10, kd=0, ki=0.1, total_distance = 25, velocity = 1):
    # create an instance of the graph class
    track = graph(0, 0, 90)

    for _ in range(100):
        track.move_forward(0.1)
    for _ in range(90):
        track.turn(-1)
        track.move_forward(np.pi / 180 * 10)
    # Plot the trajectory

    car = graph(0.1, 0, 90)
    
    num_points = 100 / velocity
    omega = 0
    error = 0
    prev_error = error

    error_sum = 0
    for _ in range(int(num_points)):
        car.theta += omega
        error = calculate_error(car, track)
        error_sum += error
        omega = kp * error + kd * (error - prev_error) + ki * error_sum
        car.move_forward(total_distance / num_points)

    plt.gca().set_aspect('equal')
    plt.grid(True)  # This activates the grid
    
    plt.plot(track.x_points, track.y_points, label="Trajectory")
    plt.plot(car.x_points, car.y_points, label="Car")
    plt.show()


In [10]:
widgets.interact(
    PID,
    kp=widgets.FloatText(value=10.0, step=0.1),
    kd=widgets.FloatText(value=0.01, step=0.1),
    ki=widgets.FloatText(value=0.01, step=0.01),
    total_distance=widgets.FloatText(value=25, step=0.1),
    velocity=widgets.FloatText(value=1, step=0.1),
)

interactive(children=(FloatText(value=10.0, description='kp', step=0.1), FloatText(value=0.01, description='kd…

<function __main__.PID(kp=10, kd=0, ki=0.1, total_distance=25, velocity=1)>